In [ ]:
import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from wildfires.analysis import corr_plot
from wildfires.data import homogenise_cube_attributes
from wildfires.utils import ensure_datetime

from data import data_dir

mpl.rc_file("matplotlibrc")

In [ ]:
[str(s) for s in list(data_dir.glob("*"))]

In [ ]:
cubes = iris.load(
    [str(s) for s in list(data_dir.glob("*.nc")) if "_day_" not in str(s)]
)
cubes

In [ ]:
sorted(
    {cube.long_name for cube in cubes}
    - {"Leaf Area Index on Land-Use Tile", "Net Primary Productivity on Land-Use Tile"}
)

In [ ]:
time_slice = slice(-80, None)

cube_names = sorted(
    {cube.long_name for cube in cubes}
    - {
        "Leaf Area Index on Land-Use Tile",
        "Net Primary Productivity on Land-Use Tile",
        "Carbon Mass in Model Soil Pool",  # Very similar to Fast Soil Pool.
        "Carbon Mass in Roots",  # Very similar to carbon mass in leaves.
    }
)

lat_lon_dict = {
    "UK": (51.5, 0),
    "Uganda": (2.36, 32.51),
    "Durban": (-29.13, 31),
    "Cape Town": (-33.57, 19.28),
    "Yosemite": (37.7, 360 - 119.67),
}

In [ ]:
fig, axes = plt.subplots(
    len(cube_names),
    1,
    figsize=(9, 2.5 * len(cube_names)),
    sharex=True,
    constrained_layout=True,
    dpi=120,
)

for ax, var in zip(axes, cube_names):
    var_cube = homogenise_cube_attributes(
        cubes.extract(iris.Constraint(name=var))
    ).concatenate_cube()[time_slice]
    ax.set_title(f"{var_cube.long_name} ({var_cube.units})")

    for (location, (lat, lon)) in lat_lon_dict.items():
        lat_i = np.argmin(np.abs(var_cube.coord("latitude").points - lat))
        lat_sel = var_cube.coord("latitude").points[lat_i]

        lon_i = np.argmin(np.abs(var_cube.coord("longitude").points - lon))
        lon_sel = var_cube.coord("longitude").points[lon_i]
        plot_data = var_cube[:, lat_i, lon_i].data.copy()
        plot_data_mean = np.mean(plot_data)
        plot_data -= plot_data_mean

        ax.plot(
            [
                ensure_datetime(var_cube.coord("time").cell(i).point)
                for i in range(var_cube.shape[0])
            ],
            plot_data,
            label=f"{location} ($\mu =${plot_data_mean:0.1e})",
            marker=".",
        )
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1))

In [ ]:
fig, axes = plt.subplots(
    len(lat_lon_dict),
    1,
    figsize=(9, 3 * len(lat_lon_dict)),
    sharex=True,
    constrained_layout=True,
    dpi=120,
)

for (ax, (location, (lat, lon))) in zip(axes, lat_lon_dict.items()):
    ax.set_title(f"{location}")

    corr_data = {}

    for var, c in zip(cube_names, plt.get_cmap("tab20").colors):
        var_cube = homogenise_cube_attributes(
            cubes.extract(iris.Constraint(name=var))
        ).concatenate_cube()[time_slice]

        lat_i = np.argmin(np.abs(var_cube.coord("latitude").points - lat))
        lat_sel = var_cube.coord("latitude").points[lat_i]

        lon_i = np.argmin(np.abs(var_cube.coord("longitude").points - lon))
        lon_sel = var_cube.coord("longitude").points[lon_i]

        # Extract data.
        plot_data = var_cube[:, lat_i, lon_i].data.copy()

        # Normalise Data.
        plot_data -= plot_data.mean()
        plot_data /= np.max(np.abs(plot_data))

        corr_data[var_cube.long_name] = plot_data

        ax.plot(
            [
                ensure_datetime(var_cube.coord("time").cell(i).point)
                for i in range(var_cube.shape[0])
            ],
            plot_data,
            label=f"{var_cube.long_name}",
            marker=".",
            c=c,
        )
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1))

    corr_plot(pd.DataFrame(corr_data), ax=plt.subplots()[1])

In [ ]:
for location, (lat, lon) in lat_lon_dict.items():

    fig, axes = plt.subplots(
        len(cube_names),
        1,
        figsize=(7, 0.8 * len(cube_names)),
        sharex=True,
        constrained_layout=True,
        dpi=120,
    )

    fig.suptitle(f"{location}")

    for ax, var in zip(axes, cube_names):
        var_cube = homogenise_cube_attributes(
            cubes.extract(iris.Constraint(name=var))
        ).concatenate_cube()[time_slice]

        ax.set_title(f"{var_cube.long_name} ({var_cube.units})")

        lat_i = np.argmin(np.abs(var_cube.coord("latitude").points - lat))
        lat_sel = var_cube.coord("latitude").points[lat_i]

        lon_i = np.argmin(np.abs(var_cube.coord("longitude").points - lon))
        lon_sel = var_cube.coord("longitude").points[lon_i]

        # Extract data.
        plot_data = var_cube[:, lat_i, lon_i].data.copy()

        ax.plot(
            [
                ensure_datetime(var_cube.coord("time").cell(i).point)
                for i in range(var_cube.shape[0])
            ],
            plot_data,
            marker=".",
        )